In [1]:
import math
import pandas as pd

In [2]:
def MPS_chase(data, initial_inventory, safety_stock):
    atp = []
    mps = []
    inventory = []

    forecast = list(data['Forecast'])
    orders = list(data['Order'])
    months = list(data['Month'])
    period = len(data)

    # MPS 및 재고 계산
    for i in range(period):
        demand = max(orders[i], forecast[i])
        if i == 0:
            initial = initial_inventory
        else:
            initial = inventory[-1]

        if initial - demand < safety_stock:
            mps.append(demand - initial + safety_stock)
        else:
            mps.append(0)

        inventory.append(initial + mps[-1] - demand)

    # ATP 계산
    for i in range(period):
        if i == 0:
            available = initial_inventory + mps[i] - orders[i]
        else:
            available = mps[i] - orders[i]

        for j in range(i + 1, period):
            if mps[j] == 0:
                available -= orders[j]
            else:
                break

        atp.append(available if mps[i] > 0 else 0)

    result = pd.DataFrame(
        data=[forecast, orders, inventory, atp, mps],
        columns=months,
        index=["Forecast", "Order", "Projected Available Balance", "Available-to-Promise", "MPS"]
    )

    print("Chase MPS")
    print("On-Hand Inventory:", initial_inventory)
    print("Safety Stock:", safety_stock)

    return result

# 예제 데이터
data = pd.DataFrame({
    'Month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'Forecast': [10, 10, 15, 10, 10, 15, 15, 15, 15, 15, 15, 15],
    'Order': [5, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
})

initial_inventory = 10
safety_stock = 0

result = MPS_chase(data, initial_inventory, safety_stock)

# 결과 출력 형식 맞추기
print(result)
print(result.transpose().to_string(header=False))

Chase MPS
On-Hand Inventory: 10
Safety Stock: 0
                             1   2   3   4   5   6   7   8   9   10  11  12
Forecast                     10  10  15  10  10  15  15  15  15  15  15  15
Order                         5   3   3   0   0   0   0   0   0   0   0   0
Projected Available Balance   0   0   0   0   0   0   0   0   0   0   0   0
Available-to-Promise          0   7  12  10  10  15  15  15  15  15  15  15
MPS                           0  10  15  10  10  15  15  15  15  15  15  15
1   10  5  0   0   0
2   10  3  0   7  10
3   15  3  0  12  15
4   10  0  0  10  10
5   10  0  0  10  10
6   15  0  0  15  15
7   15  0  0  15  15
8   15  0  0  15  15
9   15  0  0  15  15
10  15  0  0  15  15
11  15  0  0  15  15
12  15  0  0  15  15


## Paring MPS

In [3]:
paring_data = pd.read_csv('Paring_MPS.csv')
init_inventory = 12

print("[ Paring ]")
paring_mps = MPS_chase(paring_data, init_inventory, 0)
paring_mps

[ Paring ]
Chase MPS
On-Hand Inventory: 12
Safety Stock: 0


,1,2,3,4,5,6,7,8,9,10,11,12
Forecast,10,10,10,15,15,15,15,20,20,20,15,15
Order,5,2,4,0,0,0,0,0,0,0,0,0
Projected Available Balance,2,0,0,0,0,0,0,0,0,0,0,0
Available-to-Promise,0,6,6,15,15,15,15,20,20,20,15,15
MPS,0,8,10,15,15,15,15,20,20,20,15,15


## Middle MPS

In [4]:
middle_data = pd.read_csv('Middle_MPS.csv')
init_inventory = 15
print("[ Middle ]")
middle_mps = MPS_chase(middle_data, init_inventory, 0)
middle_mps

[ Middle ]
Chase MPS
On-Hand Inventory: 15
Safety Stock: 0


,1,2,3,4,5,6,7,8,9,10,11,12
Forecast,20,20,20,20,25,20,15,20,20,20,15,15
Order,15,9,12,0,0,0,0,0,0,0,0,0
Projected Available Balance,0,0,0,0,0,0,0,0,0,0,0,0
Available-to-Promise,5,11,8,20,25,20,15,20,20,20,15,15
MPS,5,20,20,20,25,20,15,20,20,20,15,15


## Engine MPS

In [5]:
engine_data = pd.read_csv('Engine_MPS.csv')
init_inventory = 20

print("[ Engine ]")
engine_mps = MPS_chase(engine_data, init_inventory, 0)
engine_mps

[ Engine ]
Chase MPS
On-Hand Inventory: 20
Safety Stock: 0


,1,2,3,4,5,6,7,8,9,10,11,12
Forecast,25,25,25,25,30,30,30,20,20,25,15,15
Order,15,8,7,6,4,0,0,0,0,0,0,0
Projected Available Balance,0,0,0,0,0,0,0,0,0,0,0,0
Available-to-Promise,10,17,18,19,26,30,30,20,20,25,15,15
MPS,5,25,25,25,30,30,30,20,20,25,15,15


In [7]:
import pandas as pd

def L4L_solver(gross_req, sched_receipts, on_hand, safety_stock, lead_time, to_print):
    global planned_order_release
    planned_order_release = []
    planned_receipts = []
    inventory = []

    sched_receipts.extend([0] * (len(gross_req) - len(sched_receipts)))

    for i in range(len(gross_req)):
        if i == 0:
            if on_hand - gross_req[0] + sched_receipts[0] < safety_stock:
                planned_receipts.append(gross_req[0] - on_hand - sched_receipts[0] + safety_stock)
            else:
                planned_receipts.append(0)
            inventory.append(on_hand - gross_req[0] + sched_receipts[i] + planned_receipts[0])
        else:
            if inventory[i-1] + sched_receipts[i] - gross_req[i] < safety_stock:
                planned_receipts.append(gross_req[i] - inventory[i-1] - sched_receipts[i] + safety_stock)
            else:
                planned_receipts.append(0)
            inventory.append(inventory[i-1] - gross_req[i] + sched_receipts[i] + planned_receipts[i])

    for _ in range(lead_time):
        planned_receipts.pop(0)
        planned_receipts.append(0)

    planned_order_release = planned_receipts

    if to_print:
        data = pd.DataFrame(
            [gross_req, sched_receipts, inventory, planned_order_release],
            columns=[i+1 for i in range(len(gross_req))],
            index=["Gross requirements", "Scheduled receipts", "Project available balance", "Planned order release"]
        )
        print("Lot for Lot")
        print(data)
        print("On Hand:", on_hand)
        print("Safety Stock:", safety_stock)
        print("Lead Time:", lead_time)
        print("\n")

def update_MPS_list(BOM, multi_BOM, MPS_list, planned_order_release_list):
    for k in range(1, len(BOM)):
        for i in range(len(BOM[k])):
            if BOM[k][i]:
                for j in range(len(MPS_list["A"])):
                    if i < len(BOM[k-1]) and BOM[k-1][i] and BOM[k-1][i] in planned_order_release_list:
                        MPS_list[BOM[k][i]][j] += multi_BOM[k][i] * planned_order_release_list[BOM[k-1][i]][j]
                if k != len(BOM) - 1:
                    if i != len(BOM[k]) - 1:
                        if BOM[k][i] not in BOM[k][i+1:] and all(BOM[m][n] != BOM[k][i] for m in range(k+1, len(BOM)) for n in range(len(BOM[m]))):
                            process_part(BOM[k][i], MPS_list, planned_order_release_list)
                    else:
                        if all(BOM[m][n] != BOM[k][i] for m in range(k+1, len(BOM)) for n in range(len(BOM[m]))):
                            process_part(BOM[k][i], MPS_list, planned_order_release_list)
                else:
                    if i != len(BOM[k]) - 1:
                        if BOM[k][i] not in BOM[k][i+1:]:
                            process_part(BOM[k][i], MPS_list, planned_order_release_list)
                    else:
                        process_part(BOM[k][i], MPS_list, planned_order_release_list, False)
                # Ensure all parts are processed
                if k == len(BOM) - 1:
                    process_part(BOM[k][i], MPS_list, planned_order_release_list)




def process_part(part, MPS_list, planned_order_release_list, to_print=True):
    print("Part:", part)
    L4L_solver(
        MPS_list[part],
        scheduled_receipts_list[part],
        on_hand_list[part],
        safety_stock_list[part],
        lead_time_list[part],
        to_print
    )
    planned_order_release_list[part] = planned_order_release

## MRP inputs

# 자재 명세서 (BOM, Bill of Materials) 트리 구조
BOM = [
    ["A","","","B","","","","C","","",""],
    ["D","E","","F","G","H","I","J","K",""],
    ["L","","M","L","N","L","N","O","P","Q","R","S"]
]

# 각 항목별 필요 수량 (BOM Multipliers)
multi_BOM = [
    [1,1,1,1,1,1,1,1,1,1,1],  # A, B, C 각 제품의 상위 레벨 필요 수량
    [2,1,0,1,1,1,2,3,1,0,0],  # D, E, F, G, H, I, J, K 각 제품의 중간 레벨 필요 수량
    [1,0,1,3,2,5,3,1,4,3,5,2]  # L, M, L, N, L, N, O, P, Q, R, S 각 제품의 하위 레벨 필요 수량
]

lead_time_list = {"A":1, "B":1, "C":1, "D":2, "E":1, "F":2, "G":2, "H":1, "I":2, \
                  "J":2,"K":1,"L":1,"M":1,"N":2,"O":1,"P":1,"Q":3,"R":1,"S":2}

scheduled_receipts_list = {
    "A":[50,20],
    "B":[45,10],
    "C":[50,10],
    "D":[0,20],
    "E":[50,15],
    "F":[70,60],
    "G":[50,30],
    "H":[40,30],
    "I":[50,10],
    "J":[25,10],
    "K":[10,10],
    "L":[10,10],
    "M":[80,60,0,0,0,60],
    "N":[50,10],
    "O":[40,20],
    "P":[0,20],
    "Q":[50,40],
    "R":[50,15],
    "S":[50,10]
}

safety_stock_list = {"A":0, "B":0, "C":0, "D":0, "E":0, "F":0, "G":0, "H":0, "I":0, "J":0,\
                    "K":0,"L":0,"M":0,"N":0,"O":0,"P":0,"Q":0,"R":0,"S":0}

on_hand_list = {"A":10, "B":15, "C":15, "D":30, "E":50, "F":30, "G":50, "H":40, "I":15, "J":15,\
                    "K":20,"L":20,"M":50,"N":15,"O":30,"P":30,"Q":10,"R":30,"S":15}

MPS_list = {
    "A": list(paring_mps.loc['MPS',:]),
    "B": list(middle_mps.loc['MPS',:]),
    "C": list(engine_mps.loc['MPS',:]),
    "D": [0 for _ in range(12)],
    "E": [0 for _ in range(12)],
    "F": [0 for _ in range(12)],
    "G": [0 for _ in range(12)],
    "H": [0 for _ in range(12)],
    "I": [0 for _ in range(12)],
    "J": [0 for _ in range(12)],
    "K": [0 for _ in range(12)],
    "L": [0 for _ in range(12)],
    "M": [0 for _ in range(12)],
    "N": [0 for _ in range(12)],
    "O": [0 for _ in range(12)],
    "P": [0 for _ in range(12)],
    "Q": [0 for _ in range(12)],
    "R": [0 for _ in range(12)],
    "S": [0 for _ in range(12)],   
}

planned_order_release_list = {part: [0 for _ in range(12)] for part in ["A", "B", "C", "D", "E", "F", "G", "H", \
                                                                        "I", "J","K","L","M","N","O","P","Q","R","S"]}

# Level 0
for i in range(len(BOM[0])):
    if BOM[0][i]:
        process_part(BOM[0][i], MPS_list, planned_order_release_list)
    for k in range(len(BOM[0])):
        if BOM[0][i] == "":
            BOM[0][i] = BOM[0][i-1]

# Level n
update_MPS_list(BOM, multi_BOM, MPS_list, planned_order_release_list)

Part: A
Lot for Lot
                           1   2   3   4   5   6   7   8   9   10  11  12
Gross requirements          0   8  10  15  15  15  15  20  20  20  15  15
Scheduled receipts         50  20   0   0   0   0   0   0   0   0   0   0
Project available balance  60  72  62  47  32  17   2   0   0   0   0   0
Planned order release       0   0   0   0   0   0  18  20  20  15  15   0
On Hand: 10
Safety Stock: 0
Lead Time: 1


Part: B
Lot for Lot
                           1   2   3   4   5   6   7   8   9   10  11  12
Gross requirements          5  20  20  20  25  20  15  20  20  20  15  15
Scheduled receipts         45  10   0   0   0   0   0   0   0   0   0   0
Project available balance  55  45  25   5   0   0   0   0   0   0   0   0
Planned order release       0   0   0  20  20  15  20  20  20  15  15   0
On Hand: 15
Safety Stock: 0
Lead Time: 1


Part: C
Lot for Lot
                           1   2   3   4   5   6   7   8   9   10  11  12
Gross requirements          5  25  25  2

In [13]:
def update_chase(f_sale, s_sale, f_production, s_production, data_list, init_inventory_list, safety_stock_list, MPS_list):
    """
    chasing 업데이트.

    Args:
        f_sale (dict): 첫 번째 달 판매 예측.
        s_sale (dict): 두 번째 달 판매 예측.
        f_production (dict): 첫 번째 달 생산 계획.
        s_production (dict): 두 번째 달 생산 계획.
        data_list (dict): 각 제품에 대한 데이터.
        init_inventory_list (dict): 각 제품의 초기 재고.
        safety_stock_list (dict): 각 제품의 안전 재고.
        MPS_list (dict): 각 제품의 마스터 생산 일정.
    """

    for key in data_list:
        data = data_list[key]
        init_inventory = init_inventory_list[key]
        safety_stock = safety_stock_list[key]

        # 기존 MPS 계산
        mps = MPS_chase(data, init_inventory, safety_stock)
        print('기존 MPS for', key)
        print(mps)
        print('\n')

        # Order 업데이트.
        data.loc[0, 'Order'] = f_sale[key]
        data.loc[1, 'Order'] = s_sale[key]

        # 수정된 MPS 계산
        print('수정된 MPS for', key)
        revised_mps = MPS_chase(data, init_inventory, safety_stock)
        print(revised_mps)
        print('\n')

        #  MPS 리스트 업데이트
        MPS_list[key][0] = f_production[key]
        MPS_list[key][1] = s_production[key]

    # planned order release 업데이트
    for part in ["A", "B", "C"]:
        process_part(part, MPS_list, planned_order_release_list)

    # 하위 구성 요소 MPS 리스트 업데이트
    update_MPS_list(BOM, multi_BOM, MPS_list, planned_order_release_list)

# 업데이트된 update_chase 함수의 예시 사용법
update_chase(
    f_sale={'A': 10, 'B': 20, 'C': 30},
    s_sale={'A': 15, 'B': 10, 'C': 20},
    f_production={'A': 20, 'B': 30, 'C': 10},
    s_production={'A': 15, 'B': 10, 'C': 10},
    data_list={'A': paring_data, 'B': middle_data, 'C': engine_data},
    init_inventory_list={'A': 10, 'B': 15, 'C': 5},
    safety_stock_list={'A': 0, 'B': 0, 'C': 0},
    MPS_list=MPS_list
)


Chase MPS
On-Hand Inventory: 10
Safety Stock: 0
기존 MPS for A
                             1   2   3   4   5   6   7   8   9   10  11  12
Forecast                     10  10  10  15  15  15  15  20  20  20  15  15
Order                        20  15   4   0   0   0   0   0   0   0   0   0
Projected Available Balance   0   0   0   0   0   0   0   0   0   0   0   0
Available-to-Promise          0   0   6  15  15  15  15  20  20  20  15  15
MPS                          10  15  10  15  15  15  15  20  20  20  15  15


수정된 MPS for A
Chase MPS
On-Hand Inventory: 10
Safety Stock: 0
                             1   2   3   4   5   6   7   8   9   10  11  12
Forecast                     10  10  10  15  15  15  15  20  20  20  15  15
Order                        10  15   4   0   0   0   0   0   0   0   0   0
Projected Available Balance   0   0   0   0   0   0   0   0   0   0   0   0
Available-to-Promise          0   0   6  15  15  15  15  20  20  20  15  15
MPS                           0  15  10